<a href="https://colab.research.google.com/github/Zoghbii/Perros-Y-Gatos/blob/main/image_classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de imágenes desde cero (gatos y Perros)

**Descripción:** Entrenamiento de un clasificador de imágenes desde cero para perros y gatos.

## Introduccion
Utilizamos la utilidad `image_dataset_from_directory` para generar los conjuntos de datos y las capas de preprocesamiento de imágenes de Keras para la estandarización y el aumento de datos.

## Setup

In [ ]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

## Cargamos los datos: el conjunto de datos Gatos vs. Perros

###
Descarga de datos sin procesar

Primero, descargamos el archivo ZIP de 786M de los datos sin procesar:

In [ ]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

In [ ]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

Ahora tenemos una carpeta (PetImages) que contiene dos (subcarpetas): Gato y Perro. Cada subcarpeta contiene archivos de imagen para cada categoría.

In [ ]:
!ls PetImages

### Filtrar imágenes corruptas

Al trabajar con una gran cantidad de datos de imágenes reales, es común encontrar imágenes corruptas. Filtraremos las imágenes corruptas que no incluyan la cadena "JFIF" en su encabezado.

In [ ]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")


## Generamos un `Conjunto de datos`

In [ ]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

## vemos los datos

Aquí están las primeras 9 imágenes en el conjunto de datos de entrenamiento.

In [ ]:

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

## aumento de datos de imágenes


Cuando no se dispone de un conjunto grande de datos de imágenes, es recomendable introducir artificialmente diversidad de muestras aplicando transformaciones aleatorias pero realistas a las imágenes de entrenamiento, como volteos horizontales aleatorios o pequeñas rotaciones aleatorias. Esto ayuda a exponer el modelo a diferentes aspectos de los datos de entrenamiento y, al mismo tiempo, ralentiza el sobreajuste.

In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images


Visualicemos cómo se ven las muestras aumentadas aplicando `data_augmentation` repetidamente a las primeras imágenes del conjunto de datos:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")


## Estandarización de los datos

Nuestras imágenes ya tienen un tamaño estándar (180x180), ya que nuestro conjunto de datos las genera como lotes contiguos de `float32`. Sin embargo, sus valores de canal RGB están en el rango `[0, 255]`. Esto no es ideal para una red neuronal; en general, se recomienda que los valores de entrada sean pequeños. Aquí, estandarizaremos los valores para que estén en el rango `[0, 1]` mediante una capa de `Reescalado` al inicio de nuestro modelo

## Configurar el conjunto de datos para el rendimiento

Apliquemos la ampliación de datos a nuestro conjunto de datos de entrenamiento y asegurémonos de usar la precarga en búfer para poder extraer datos del disco sin que la I/O se bloquee.

In [ ]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

## Construir un modelo

Construiremos una versión reducida de la red Xception. No hemos intentado optimizar la arquitectura en detalle; si desea realizar una búsqueda sistemática de la mejor configuración del modelo, considere usar [KerasTuner](https://github.com/keras-team/keras-tuner).

Tenga en cuenta que:

- Iniciamos el modelo con el preprocesador `data_augmentation`, seguido de una capa `Rescaling`.
- Incluimos una capa `Dropout` antes de la capa de clasificación final.

In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

## Entrenando el Modelo

In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)


Alcanzamos una precisión de validación superior al 90 % después de entrenar durante 25 épocas con el conjunto de datos completo (en la práctica, se puede entrenar durante más de 50 épocas antes de que el rendimiento de la validación comience a disminuir).


## Ejecutar inferencia con datos nuevos

debemos tener en cuenta que el aumento y la eliminación de datos están inactivos durante la inferencia.

In [ ]:
img = keras.utils.load_img("PetImages/Cat/6779.jpg", target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")